<a href="https://colab.research.google.com/github/igedeaguss/Magang-Pharos/blob/main/Coba_logic_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rapidfuzz
import pandas as pd
import re
from rapidfuzz import fuzz, process

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.5 MB/s eta 0:00:00


In [2]:
data_ims = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/IMS Compiled Q3 2019 - Q2 2024 (List Product).xlsx', sheet_name='Sheet1')
data_survey = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Hospital Survey Product Name Pivot.xlsx', sheet_name='Sheet1')

data_prdk_survey = data_survey.copy()
data_ims_prdk = data_ims.copy()

In [3]:
import re
import pandas as pd

def clean_product_name(name):
    return name.split(' -')[0].lower()

def clean_name_survey(name):
  # Hapus bagian setelah tanda "-" jika diikuti oleh karakter lain (misalnya "HARVEST COUGH - ADD" -> "HARVEST COUGH")
    name = re.sub(r"\s*-.*$", "", name)

    # Hapus karakter khusus tetapi tetap menjaga format yang benar
    name = re.sub(r'[\[\]{}@]', '', name)  # Hapus tanda [], {}, dan @
    name = re.sub(r'(?<!\s)-(?<!\s)', ' ', name)  # Ganti "-" yang tidak diapit spasi dengan spasi
    name = re.sub(r'[()]', '', name)  # Hapus tanda kurung biasa tetapi biarkan isinya

    # Hilangkan spasi ganda yang mungkin muncul akibat penghapusan
    name = re.sub(r'\s+', ' ', name).strip()

    return name.lower()

data_ims_prdk['Cleaned Brand Name'] = data_ims_prdk['Product'].apply(clean_product_name)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Product Name'].apply(clean_name_survey)

In [11]:
abbotic_survey = data_prdk_survey[data_prdk_survey['Cleaned Name'].str.contains(fr"\b{re.escape('ABBOTIC')}\b", case=False, na=False)]
abbotic_ims = data_ims_prdk[data_ims_prdk['Cleaned Brand Name'].str.contains(fr"\b{re.escape('ABBOTIC')}\b", case=False, na=False)]

In [29]:
import pandas as pd
import re
from rapidfuzz import fuzz

# Fungsi untuk mengekstrak angka dan unit dari nama produk
def extract_numbers_and_units(name):
    matches = re.findall(r'(\d+\.?\d*)\s?([A-Za-z]*)', name)  # Tangkap angka + unit
    return {f"{num} {unit}".strip() for num, unit in matches}  # Gunakan set agar unik

# Fungsi untuk mencocokkan angka dengan unit
def match_numbers_with_units(survey_numbers, pack_numbers):
    if not survey_numbers or not pack_numbers:
        return 0  # Jika salah satu kosong, skor = 0

    survey_set = set(survey_numbers)
    pack_set = set(pack_numbers)
    intersection = survey_set.intersection(pack_set)

    # Skor dihitung berdasarkan banyaknya pasangan angka + satuan yang cocok
    score = (len(intersection) / max(len(survey_set), len(pack_set))) * 100
    return score

# Fungsi untuk mencocokkan bentuk sediaan
def match_dosage_form(survey_str, ims_str):
    common_forms = ["SYRUP", "SYR DRY", "TAB", "CAP", "INJ", "POWDER"]

    # Temukan bentuk sediaan dalam kedua string
    survey_form = next((form for form in common_forms if form in survey_str.upper()), "")
    ims_form = next((form for form in common_forms if form in ims_str.upper()), "")

    # Jika bentuk sediaan ada di kedua produk, gunakan fuzzy matching
    if survey_form and ims_form:
        return fuzz.partial_ratio(survey_form, ims_form)

    return 50  # Jika tidak ada bentuk sediaan, beri skor default rendah

# Fungsi untuk menghitung skor kemiripan dengan bobot yang lebih baik
def calculate_similarity(produk_survey, pack_list):
    results = []
    survey_numbers = extract_numbers_and_units(produk_survey)

    for pack in pack_list:
        pack_numbers = extract_numbers_and_units(pack)

        # Hitung skor kemiripan angka + satuan
        score_numbers = match_numbers_with_units(survey_numbers, pack_numbers)

        # Hitung skor kemiripan string (nama produk)
        score_strings = fuzz.token_sort_ratio(produk_survey, pack)

        # Hitung kemiripan bentuk sediaan
        score_dosage_form = match_dosage_form(produk_survey, pack)

        # Gabungkan skor dengan bobot (angka lebih penting)
        combined_score = (0.6 * score_numbers) + (0.3 * score_strings) + (0.1 * score_dosage_form)
        results.append((pack, combined_score))

    return results

# Fungsi utama untuk memproses data
def process_brand_matching(data_ims, data_survey):
    results = []
    unique_brands = data_ims['Cleaned Brand Name'].unique()

    for brand in unique_brands:
        # Filter langsung berdasarkan "Cleaned Brand Name"
        subset_ims = data_ims[data_ims['Cleaned Brand Name'] == brand]

        # Filter produk survey yang mengandung nama brand
        subset_survey = data_survey[data_survey['Product Name'].str.contains(fr"\b{re.escape(brand)}\b", case=False, na=False, regex=True)]

        for _, survey_row in subset_survey.iterrows():
            produk_survey = survey_row['Product Name']
            pack_list = subset_ims['Pack'].tolist()

            # Menghitung skor kemiripan
            similarity_scores = calculate_similarity(produk_survey, pack_list)

            # Cari produk IMS dengan skor tertinggi
            if similarity_scores:
                best_match_pack, best_score = max(similarity_scores, key=lambda x: x[1])

                # Simpan hasil terbaik
                results.append({
                    'Brand': brand,
                    'Product Survey': produk_survey,
                    'Best Match IMS Pack': best_match_pack,
                    'Best Similarity Score': best_score
                })

    # Buat DataFrame hasil
    results_df = pd.DataFrame(results)
    return results_df.sort_values(by='Best Similarity Score', ascending=False)

# Contoh penggunaan
filtered_results = process_brand_matching(abbotic_ims, abbotic_survey)
filtered_results


,Brand,Product Survey,Best Match IMS Pack,Best Similarity Score
15,abbotic,ABBOTIC 250MG/5ML DRY SYR 50ML,ABBOTIC SYR DRY 250MG/5ML 50ML,95.000000
8,abbotic,ABBOTIC 125MG/5ML DRY SYR 30ML,ABBOTIC SYR DRY 125MG/5ML 30ML,95.000000
10,abbotic,ABBOTIC 125MG/5ML-60ML DRY SYR,ABBOTIC SYR DRY 125MG/5ML 60ML,94.000000
38,abbotic,ABBOTIC XL 500 MG TAB \10,ABBOTIC TAB XL 500MG 10,92.500000
11,abbotic,ABBOTIC 125MG/5ML60ML DRYSIRU,ABBOTIC SYR DRY 125MG/5ML 60ML,92.457627
32,abbotic,ABBOTIC SYRUP (125MG/5ML) 60 ML,ABBOTIC SYR DRY 125MG/5ML 60ML,92.090164
34,abbotic,ABBOTIC SYRUP (250MG/5ML) 50 ML,ABBOTIC SYR DRY 250MG/5ML 50ML,92.090164
33,abbotic,ABBOTIC SYRUP (125MG/5ML) 30 ML,ABBOTIC SYR DRY 125MG/5ML 30ML,92.090164
9,abbotic,ABBOTIC 125MG/5ML30ML DRYSIRUP,ABBOTIC SYR DRY 125MG/5ML 30ML,92.000000
4,abbotic,ABBOTIC 125 MG/5 ML 60 ML DRY SYRUP,ABBOTIC SYR DRY 125MG/5ML 60ML,87.807692
